In [30]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [31]:
os.listdir('/kaggle/input/satellite-image-classification/data/')

In [32]:
print(len(os.listdir('../input/satellite-image-classification/data/cloudy/')))
print(len(os.listdir('../input/satellite-image-classification/data/desert/')))
print(len(os.listdir('../input/satellite-image-classification/data/green_area/')))
print(len(os.listdir('../input/satellite-image-classification/data/water/')))

In [33]:
import matplotlib.pyplot as plt
x=['Cloudy','Desert','Green area','Water']
y=[1500,1131,1500,1500]
plt.bar(x,y);

In [34]:
import tensorflow 
from keras.models import Sequential 
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator as IDG, load_img 
from tensorflow.keras.preprocessing import image_dataset_from_directory

from IPython import display 
display.set_matplotlib_formats('svg')

In [35]:
train_datagen = IDG(rescale=1./255,
                   fill_mode='nearest',
                   featurewise_center=True,
                   validation_split=0.25,
                   horizontal_flip=True,
                   shear_range=0.3,
                   zoom_range=0.25,
                   brightness_range=(0.4,0.7),
                   rotation_range=15,
                   featurewise_std_normalization=True,
                   )

In [36]:
train_generator= train_datagen.flow_from_directory('../input/satellite-image-classification/data',
                                                  target_size=(224,224),
                                                  batch_size=32,
                                                  class_mode='binary',
                                                  subset='training')

In [37]:
test_generator = train_datagen.flow_from_directory('../input/satellite-image-classification/data',
                                                 target_size=(224,224),
                                                 batch_size=32,
                                                 class_mode='binary',
                                                 subset='validation')

In [38]:
from keras.applications.mobilenet_v2 import MobileNetV2,preprocess_input

In [39]:
model = Sequential()
base_model = MobileNetV2(input_shape = [224, 224, 3], weights = 'imagenet', include_top = False)

In [40]:
base_model.trainable = True  

for i in base_model.layers[:100]: 
    base_model.trainable = False
    
pool = GlobalAveragePooling2D()
mid_layer = Dense(100,activation = 'relu')
final_1 = Dense(4,activation = 'softmax')

model = Sequential([base_model, mid_layer,pool, final_1])
model.summary()

In [41]:
early = tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [42]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [43]:
FIT =model.fit(train_generator,
    validation_data = test_generator, 
    callbacks = [early],
    epochs = 20)

In [44]:
model.evaluate(test_generator)